In [1]:
import pandas as pd
import os
import numpy as np
import scipy

In [2]:
files = os.listdir('results_server/')
filename = "results_merged/results_2024_05_24"

In [3]:
# MERGING RESULTS
files = os.listdir('results_server/')
dfs = [
    pd.read_pickle('results_server/{}'.format(f))
    for f in files
    if f[-3:] == 'pkl'
]
df = pd.concat(dfs, ignore_index=True)
df.sort_values(by=['H']).to_csv(filename+".csv")
df.to_pickle(filename+".pkl")

In [4]:
df = pd.read_pickle(filename+".pkl")

In [5]:
df[df["H"]==0.25].sort_values(by=["res"])

,dscrt_train,dscrt,N,H,space,nn_hidden,kappa,kappa_T,sigma,initial,...,epochs,max_gb,MC_,sig_comp,save_model,res,err,time,train_time,eval_time
59,100,100,5,0.25,log,2,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.487299,0.003919,2024-05-23 22:07:31,4360.878346,14.927224
17,100,100,5,0.25,sig,0,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.474687,0.003970,2024-05-23 21:50:51,3216.166658,17.256055
70,100,100,4,0.25,log,2,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.474041,0.003948,2024-05-23 20:54:38,3096.819924,8.002649
54,100,100,4,0.25,sig,0,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.466424,0.003904,2024-05-23 20:42:35,2224.403303,10.445088
35,100,100,3,0.25,log,2,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.451988,0.003964,2024-05-23 20:05:08,3279.986100,7.865438
29,100,100,3,0.25,sig,0,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.445067,0.004001,2024-05-23 19:37:42,1500.230585,6.776317
40,100,100,2,0.25,log,2,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.306026,0.004195,2024-05-23 19:17:59,2994.289453,7.839028
37,100,100,2,0.25,sig,0,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.297740,0.004218,2024-05-23 18:53:03,1413.261834,4.940678
60,100,100,1,0.25,log,2,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,-0.010422,0.003477,2024-05-23 18:17:17,2449.090202,6.031278
55,100,100,1,0.25,sig,0,0.001,0.1,0.02,1.0,...,31,128,131072,tX,True,0.000573,0.003493,2024-05-23 17:50:39,974.846714,3.612362


In [8]:
# H in collumns's

discrt = 100
mode = ['tX']
space = ['sig', 'log']
N_range = [1,2,3,4,5]
#H_range = [.1,.2,.3,.5,.8,1.0]
H_range = [1.0/16,1.0/8,1.0/4,1.0/2,3.0/4,1.0]

df_ = df.query('dscrt == @discrt and N in @N_range and sig_comp in @mode and space in @space')
df_ = df_.sort_values(by=['sig_comp','N', 'H'], ascending=[False, True, True])

best_values = { 
    H: df_.query("H == @H")["res"].dropna().to_numpy().min().round(3)
    for H in H_range
}

# First row
print('  &  $H$', end=' ')
for H in H_range:
    print('&  {}'.format(H), end=' ')
print(' \\\\')


for spc in space:
    print('\\hline')
    print(r"\multirow{{ {} }}{{*}}{{ $\mathcal{{A}}_{{ {} }}$ }}".format(len(N_range), spc), end='')
    for N in N_range:
        print('& $N = {}$ '.format(N), end=' ')

        for H in H_range:
            res = df_.query("N == @N and space == @spc and H == @H")[["res", "err"]]
                
            if len(res) == 0:
                print('&      ', end=' ')
            else:
                v = res["res"].to_numpy()[0]
                if v.round(3) == best_values[H]:
                    print('& \\textbf{{ {:0.3f} }}'.format(-v), end=' ')
                else:
                    print('& {:0.3f}'.format(-v), end=' ')
        
        print(' \\\\ ')

  &  $H$ &  0.0625 &  0.125 &  0.25 &  0.5 &  0.75 &  1.0  \\
\hline
\multirow{ 5 }{*}{ $\mathcal{A}_{ sig }$ }& $N = 1$  & 0.032 & 0.018 & -0.001 & -0.007 & -0.001 & 0.011  \\ 
& $N = 2$  & 1.356 & 0.842 & 0.298 & 0.001 & 0.162 & 0.532  \\ 
& $N = 3$  & 2.470 & 1.437 & 0.445 & -0.001 & 0.193 & 0.674  \\ 
& $N = 4$  & 2.545 & 1.503 & 0.466 & -0.003 & 0.200 & 0.667  \\ 
& $N = 5$  & 2.585 & 1.538 & 0.475 & -0.005 & 0.202 & 0.667  \\ 
\hline
\multirow{ 5 }{*}{ $\mathcal{A}_{ log }$ }& $N = 1$  & 0.034 & 0.028 & 0.010 & -0.003 & 0.016 & 0.791  \\ 
& $N = 2$  & 1.388 & 0.873 & 0.306 & \textbf{ 0.005 } & 0.186 & 0.807  \\ 
& $N = 3$  & 2.539 & 1.471 & 0.452 & -0.003 & 0.204 & \textbf{ 0.825 }  \\ 
& $N = 4$  & 2.617 & 1.540 & 0.474 & -0.004 & \textbf{ 0.212 } & 0.807  \\ 
& $N = 5$  & \textbf{ 2.670 } & \textbf{ 1.578 } & \textbf{ 0.487 } & -0.005 & 0.205 & 0.797  \\ 


In [8]:
discrt = 100
space = ['sig', 'log']
Ns = [1, 2, 3, 4]

df_ = df.query('dscrt == @discrt and N in @Ns and space in @space')
df_ = df_.sort_values(by=['sig_comp', 'N'], ascending=[False, True])

for H in [-1, 1/4, 1/3, 1/2, 0.7, 0.9, 1.0]:
    
    if H >= 0:
        filt = df_.query("H == {}".format(H))[["H", "N", "res", "err", "space"]]
        values = filt["res"].to_numpy().round(3)
        v_min = filt["res"].dropna().to_numpy().min().round(4)
        if H == 1/3:
            print("1/3", end=' ')
        elif H == 1.0:
            print("1", end=' ')
        elif H > 1/2:
            print("{}/{}".format(int(H*10), 10), end=' ')
        else:
            print("{}/{}".format(*H.as_integer_ratio()), end = ' ')
        

    for spc in space:
        for N in Ns:
            if H == -1:
                print('& $N = {}$ '.format(N), end=' ')
            else:
                res = filt.query("N == @N and space == @spc")["res"]
                if len(res) == 0:
                    print('&      ', end=' ')
                else:
                    v = res.to_numpy()[0].round(4)
                    v_rel = -v #100*(-v - twap)/twap 
                    if v == v_min:
                        print('& \\textbf{{ {:0.3f} }}'.format(v_rel), end=' ')
                    else:
                        print('& {:0.3f}'.format(v_rel), end=' ')
    if False:
        if H == -1:
            print('& bm ', end=' ')
        elif H == 1/3:
            print('& {:0.3f} '.format(0.4144001982160647), end=' ')
        elif H == 0.7:
            print('& {:0.3f} '.format(2.1403790882061484), end=' ')
                        
    print(' \\\\ ')

& $N = 1$  & $N = 2$  & $N = 3$  & $N = 4$  & $N = 1$  & $N = 2$  & $N = 3$  & $N = 4$   \\ 
1/4 & 0.708 & 29.816 & 44.379 & \textbf{ 44.724 } & 1.120 & 30.420 & 35.701 & 37.984  \\ 
1/3 & 0.276 & 11.691 & 16.063 & \textbf{ 16.087 } & 0.471 & 11.944 & 13.142 & 13.082  \\ 
1/2 & 0.005 & 0.002 & 0.009 & \textbf{ 0.025 } & -0.016 & 0.011 & -0.040 & -0.004  \\ 
7/10 & 0.210 & 10.918 & \textbf{ 13.128 } & 12.910 & 1.048 & 11.197 & 11.826 & 11.470  \\ 
9/10 & 0.777 & 36.820 & \textbf{ 45.518 } & 44.799 & 8.810 & 39.988 & 40.322 & 40.418  \\ 
1 & 1.105 & 53.220 & 68.073 & 74.267 & 71.620 & 73.737 & 74.919 & \textbf{ 75.497 }  \\ 


In [18]:
100 * 3*10**(-7) / twap

3.0029732408325072e-05

In [19]:
bench03 = 1.0031498
bench07 = 1.0203925

100 * (bench03 - twap) / twap, 100 * (bench07 - twap) / twap

(0.4144001982160647, 2.1403790882061484)

# TWAP

In [21]:
import torch

from signature_control.examples.control.optimal_execution.run import loss_fn, rde_model, new_batch, params, update_params
from signature_control.examples.control.strategies import TimeWeightedAverage
from signature_control.examples.control.run_environment import run

In [17]:
x0 = 1.0
q0 = 1.0
T = 1.0
kappa = 0.001
kappa_T = 0.1

twap = x0*q0 - q0**2 * kappa * kappa_T / (kappa + T*kappa_T)
alpha = q0 * T * kappa_T / (kappa + T*kappa_T)
twap, alpha

(0.999009900990099, 0.9900990099009901)

In [51]:
import torch

In [56]:
a = torch.linspace(0, 1, 30).reshape(10,3)

a.mean(dim=1)

tensor([0.0345, 0.1379, 0.2414, 0.3448, 0.4483, 0.5517, 0.6552, 0.7586, 0.8621,
        0.9655])

In [3]:
import iisignature
import numpy as np

In [9]:
X = np.random.randn(1000, 100, 2)

In [11]:
iisignature.sig(X, 1).shape

(1000, 2)

In [8]:
import torch

('self',
 'params',
 'lr',
 'max_iter',
 'max_eval',
 'tolerance_grad',
 'tolerance_change',
 'history_size',
 'line_search_fn',
 'defaults')